<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/torch/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [22]:
!git clone https://github.com/DreRnc/ExplainingExplanations.git
%cd ExplainingExplanations
!pip install -r requirements.txt

Cloning into 'ExplainingExplanations'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 59 (delta 19), reused 40 (delta 12), pack-reused 0
Receiving objects: 100% (59/59), 13.46 MiB | 11.62 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/ExplainingExplanations/ExplainingExplanations


# 1.0 Preparation


## 1.1 Loading Dataset

In [23]:
from datasets import load_dataset

dataset = load_dataset("esnli")

In [24]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

Shape of training_set:  (549367, 6)
Shae of validation_set:  (9842, 6)
Shape of test_set:  (9824, 6)


In [25]:
training_set[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [26]:
n_train = n_valid = n_test = 5000

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

Shape of train_small:  (5000, 6)
Shape of valid_small:  (5000, 6)
Shape of test_small:  (5000, 6)


## 1.2 Loading T5 Model

In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Test **zero-shot** on a random task.

In [28]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=100))

Bonjour Dre, je pense que la version anglaise est bonne pour nous.


## 1.3 Zero-shot example to Verify Everything is Working

In [29]:
from src.utils import generate_prompt_mnli

In [30]:
example = training_set[0]
example

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [31]:
prompt = generate_prompt_mnli(example)
prompt

'mnli hypothesis: A person is training his horse for a competition. premise: A person on a horse jumps over a broken down airplane.'

In [32]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

neutral


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## 1.4  Tokenize the dataset

In [33]:
train_small.info

DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None), 'explanation_1': Value(dtype='string', id=None), 'explanation_2': Value(dtype='string', id=None), 'explanation_3': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='esnli', config_name='plain_text', version=0.0.2, splits={'train': SplitInfo(name='train', num_bytes=107611996, num_examples=549367, shard_lengths=None, dataset_name='esnli'), 'validation': SplitInfo(name='validation', num_bytes=3416319, num_examples=9842, shard_lengths=None, dataset_name='esnli'), 'test': SplitInfo(name='test', num_bytes=3379781, num_examples=9824, shard_lengths=None, dataset_name='esnli')}, download_checksums={'hf://datasets/esnli@f01d1430c0ad65a3785d06079a0e01e015e769b8/plain_text/

In [34]:
train_small[0]["prompt"] = "hello"

train_small[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [35]:
train_small.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None)}

In [36]:
from functools import partial
from src.utils import tokenize_function
import time

In [37]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [38]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch")
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch")
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch")
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Time taken to tokenize:  0.10033464431762695
Shape of train_small_tokenized:  (5000, 9)
Shape of valid_small_tokenized:  (5000, 9)
Shape of test_small_tokenized:  (5000, 9)


In [39]:
train_small_tokenized = train_small_tokenized.rename_column("label", "labels")
valid_small_tokenized = valid_small_tokenized.rename_column("label", "labels")
test_small_tokenized = test_small_tokenized.rename_column("label", "labels")

ValueError: New column name labels already in the dataset. Please choose a column name which is not already in the dataset. Current columns in the dataset: ['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3', 'input_ids', 'attention_mask', 'labels']

In [40]:
train_small_tokenized.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [41]:
from torch.utils.data import DataLoader

In [42]:
train_dataloader = DataLoader(train_small_tokenized, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(valid_small_tokenized, batch_size=8)
test_dataloader = DataLoader(test_small_tokenized, batch_size=8)

# 2.0 Task 1: Zero-shot evaluation

In [43]:
from src.utils import evaluate_output_mnli
from tqdm import tqdm
import torch

In [50]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

device(type='cuda')

In [54]:
correct = 0
seen = 0

for tokenized_batch in tqdm(test_dataloader):
    tokenized_batch.to(device)
    input_ids = tokenized_batch['input_ids']
    outputs = model.generate(input_ids)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    correct += evaluate_output_mnli(output, tokenized_batch['labels'])

print("Accuracy zero shot on test set: ", correct/n_test)


  0%|          | 0/625 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'to'

In [ ]:
outputs.size()

In [ ]:
output

In [ ]:
# correct = 0
# seen = 0

# for datapoint in tqdm(test_small):
# prompt = generate_prompt_mnli(datapoint)
# input_ids = tokenizer(prompt, return_tensors= "pt").input_ids
# outputs = model.generate(input_ids)
# output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# correct += evaluate_output_mnli(output, datapoint['label'])

# print("Accuracy zero shot on test set: ", correct/n_test)

# 3.0 Task 2: Fine tuning without explanations

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", evaluation_strategy="epoch"
)

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=test_small_tokenized,
    compute_metrics=compute_metrics,
)

In [ ]:
train_small_tokenized["label"].size()

In [ ]:
train_small_tokenized["input_ids"].size()

In [ ]:
trainer.train()

# 4.0 Task 4: Making the model generate explanations